In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('../input/sms-spam-collection-dataset'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

../input/sms-spam-collection-dataset/spam.csv


In [2]:
df=pd.read_csv('../input/sms-spam-collection-dataset/spam.csv', encoding='latin',usecols=["v1","v2"])

In [3]:
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
import nltk
from nltk.corpus import stopwords
import re
import string

In [5]:
#data_not_punc=[]
#for message in df.v2:
  #  message_not_punc = []
   # for punctuation in message:
    #    if punctuation not in string.punctuation:
     #       message_not_punc.append(punctuation)
    # Join the characters again to form the string.
   # message_not_punc = ''.join(message_not_punc)  
   # data_not_punc.append(message_not_punc)

In [6]:
# Remove Emails
#data_not_punc = [re.sub('\S*@\S*\s?', '', sent) for sent in data_not_punc]
 
# Remove new line characters
#data_not_punc = [re.sub('\s+', ' ', sent) for sent in data_not_punc]
 
# Remove  quotes
#data_not_punc = [re.sub("\'", "", sent) for sent in data_not_punc]
#remove links:
#data_not_punc = [re.sub(r'http\S+','', sent)for sent in data_not_punc]
#data_not_punc = [re.sub(r'www\S+','', sent)for sent in data_not_punc]
#data_not_punc = [re.sub(r'.com\S+','', sent)for sent in data_not_punc]
#data_not_punc = [re.sub(r'[0-9]','', sent)for sent in data_not_punc]

In [7]:
#tokenizer = nltk.RegexpTokenizer(r"\w+")

In [8]:
#corpus=[tokenizer.tokenize(mail) for mail in data_not_punc]

In [9]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from','subject', 're', 'edu', 'use','a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z'])


In [10]:
def transform_email(email):
    message_not_punc = [] # Message without punctuation
    for punctuation in email:
        if punctuation not in string.punctuation:
            message_not_punc.append(punctuation)
    # Join words again to form the string.
    message_not_punc = ''.join(message_not_punc) 
    # Remove Emails
    message_not_punc=re.sub('\S*@\S*\s?', '', message_not_punc)
    # Remove new line characters
    message_not_punc=re.sub('\s+', ' ', message_not_punc)
    # Remove  quotes
    message_not_punc=re.sub("\'", "", message_not_punc)
    #remove links:
    message_not_punc=re.sub(r'http\S+','', message_not_punc)
    message_not_punc=re.sub(r'www\S+','', message_not_punc)
    message_not_punc=re.sub(r'.com\S+','', message_not_punc)
    message_not_punc=re.sub(r'[0-9]','', message_not_punc)
    # Remove any stopwords for message_not_punc, but first we should     
    # to transform this into the list.
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    message_clean = list(message_not_punc.split(" "))
    message_clean=tokenizer.tokenize(message_not_punc)
    filtered_mail=[w for w in message_clean if not w.lower() in stop_words]
    return  filtered_mail

In [11]:
df['v2'].head(5).apply(transform_email)

0    [Go, jurong, point, crazy, Available, bugis, g...
1                          [Ok, lar, Joking, wif, oni]
2    [Free, entry, wkly, win, FA, Cup, final, tkts,...
3                 [dun, say, early, hor, already, say]
4    [Nah, dont, think, goes, usf, lives, around, t...
Name: v2, dtype: object

In [12]:
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [13]:
train_headlines_sentences=list(df['v2'])

In [14]:
vectorizer = CountVectorizer(analyzer = 'word' )
x_counts = vectorizer.fit_transform(train_headlines_sentences)
transformer = TfidfTransformer(smooth_idf=False)
X_tfidf = transformer.fit_transform(x_counts)

In [15]:
print(X_tfidf.shape)

(5572, 8672)


In [34]:
from sklearn.model_selection import train_test_split,GridSearchCV,KFold,cross_val_score

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, df['v1'], test_size=0.20, random_state = 50)    

In [36]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

In [38]:
parameters = {  
'alpha': (1, 0.1, 0.01, 0.001, 0.0001, 0.00001)
} 

In [39]:
cv = KFold(n_splits=10, random_state=1, shuffle=True)

In [40]:
nbModel_grid = GridSearchCV(estimator=MultinomialNB(), param_grid=parameters,cv=cv, verbose=1, n_jobs=-1)

In [41]:
nbModel_grid.fit(X_train, y_train)

Fitting 10 folds for each of 6 candidates, totalling 60 fits


GridSearchCV(cv=KFold(n_splits=10, random_state=1, shuffle=True),
             estimator=MultinomialNB(), n_jobs=-1,
             param_grid={'alpha': (1, 0.1, 0.01, 0.001, 0.0001, 1e-05)},
             verbose=1)

In [42]:
print(nbModel_grid.best_estimator_)

MultinomialNB(alpha=0.1)


In [43]:
model =MultinomialNB(alpha=0.1)

In [24]:
model.fit(X_train, y_train)

MultinomialNB(alpha=0.1)

In [25]:
y_pred = model.predict(X_test)

In [26]:
y_test_pred_model = model.predict(X_test)

In [27]:
print(metrics.accuracy_score(y_test, y_test_pred_model))
print(metrics.confusion_matrix(y_test, y_test_pred_model))

0.9766816143497757
[[947   7]
 [ 19 142]]


In [28]:
from sklearn.linear_model import LogisticRegression

In [29]:
logreg=LogisticRegression()

In [30]:
logreg.fit(X_train,y_train)

LogisticRegression()

In [31]:
y_pred = logreg.predict(X_test)

In [32]:
y_test_pred_model=logreg.predict(X_test)

In [33]:
print(metrics.accuracy_score(y_test, y_test_pred_model))
print(metrics.confusion_matrix(y_test, y_test_pred_model))

0.9560538116591928
[[954   0]
 [ 49 112]]
